# Tutorial 5 - Gaussian measures

A Gaussian measure on a Hilbert space, $X$, generalised the familar Gaussian distribution on finite-dimensionsl Euclidean spaces. Such a measure, $\mu$, is characterised by its **expectation**, this being an element of $X$, and its **covariance** which is a self-adjoint and trace-class linear operator on $X$. We use the notation $\mathcal{N}_{X}(\overline{u}, Q)$ to denote a Gaussian measure on $X$ with expectation $\overline{u} \in X$ and covariance $Q \in \mathrm{Hom}(X)$. Within ```pygeoinf``` Gaussian measures are implemented withint the ```GaussianMeasure``` class. 

